# Source Modification Example

A common source code modification that you may want to do is to output a **new variable** that is not defined in CESM. 

As an example, we will add a new variable to CAM: the atmopsheric **temperature at 750hPa**.

## Adding a variable to CAM
CAM has a history field that corresponds to the **temperature at 500hPa** and a number of other pressure levels, but not at **750hPa**. Suppose you wanted to output the **temperature at 750hPa**. The following two
calls are required to add an output variable:
```
call addfld(’T750’,...) (Add a field to the master field list)
call outfld(’T750’,...) (Collect values for this field and write to history file)
```

Each of these are now described in more detail:

## ``addfld``
The sub-routine ``addfld`` adds a field to the master list with the following syntax:
```
addfld(fname,type,avgflag,units,long name)
```
where

- **``fname``** = field name
- **``type``** = the type of field. The entry for a single level field would be “horiz only” and the entry
for a 3D field would be “(/ ’lev’ /)”.
- **``avgflag``** = Averaging flag, A = average, I=instantaneous
- **``units``** = the units of the field
- **``long name``** = Field full name

Appropriate values of these parameters for the output of T750 are:

```
call addfld(’T750’,horiz only, ’A’, ’K’, ’Temperature at 750hPa pressure surface’)
```




## ``outfld``
The subroutine ``outfld`` accumulates (or takes the minimum or, maximum of etc as appropriate) the field into the history buffer for the appropriate history tape with the following syntax
```
    outfld(fname,type,avgflag,units,long name)
```

with
- **``fname``** = Field name
- **``field``** = array containing field values
- **``idim``** = longitude dimension of field array
- **``c``** = chunk (physics) or latitude (dynamics) index.

For example:

```
    call outfld(’T750’,t750, pcols, lchnk)
```


## ``add_default`` 
Another useful subroutine is the subroutine ``add_default``. The subroutine ``add_default`` Add a field to the list of default fields on history file with the following syntax
```
    subroutine add_default (fname, tindex, flag)
```
where
- **``fname``** = Field name
- **``tindex``** = history tape index
- **``flag``** = Averaging flag: A = average (default). I = instantaneous


For example:

```
    call add_default ('T500', 1, ' ')
```